In [ ]:
import ast
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn import linear_model
import sklearn
import numpy
import random
import gzip
import math
import nltk
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def assertFloat(x): # Checks that an answer is a float
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [ ]:
f = open("stupid.json")
dataset = []
for l in f:
    dataset.append(json.loads(l))

In [ ]:
df = pd.read_json('stupid.json', lines=True)

In [ ]:
dataset[0]

{'questionType': 'yes/no',
 'asin': '8805002666',
 'answerTime': 'Mar 29, 2014',
 'unixTime': 1396076400,
 'question': 'Will this fit a gazebo with the hypotenuse/pitch (peak to corner) of around 80"?',
 'answerType': '?',
 'answer': 'I MEASURED AND IT IS 80".'}

In [ ]:
def accuracy(predictions, y):
  correct = 0
  for i in range(len(y)):
    if predictions[i] == y[i]:
      correct += 1
  return correct / len(predictions)

In [ ]:
#Exploratory Analysis

In [ ]:
df['questionType'].describe()

count          59595
unique             2
top       open-ended
freq           33248
Name: questionType, dtype: object

In [ ]:
df['answerType'].value_counts()

?    14025
Y     8548
N     3774
Name: answerType, dtype: int64

In [ ]:
for i in dataset:
  if i['asin'] == 'B003MOD2HY':
    print(i)

{'questionType': 'open-ended', 'asin': 'B003MOD2HY', 'answerTime': 'Nov 18, 2012', 'unixTime': 1353225600, 'question': 'Does this reconstitute to a powdery or fibrous material? Thanks.', 'answer': 'I have only used a little for my worms but what I have used had a texture like coffee grains when I used water to reconstitute it. I like the stuff.'}
{'questionType': 'open-ended', 'asin': 'B003MOD2HY', 'answerTime': 'Feb 17, 2015', 'unixTime': 1424160000, 'question': 'when reconstited i know it expands some but how well does it hold together? or does it fall apart?', 'answer': 'once reconstituted it falls apart and acts more like a soil.'}
{'questionType': 'open-ended', 'asin': 'B003MOD2HY', 'answerTime': 'Mar 6, 2014', 'unixTime': 1394092800, 'question': 'Has anybody used this to make a potting mix for citrus? What was your mix "recipe"?', 'answer': 'You only need 20% mixed with proper soil and an acceptable climate. And mix the 20% with 20% soil right at the spot where the roots will tou

In [ ]:
# Tf-idf

In [ ]:
wordCount = defaultdict(int)
totalRead =0

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
9*len(dataset)//10

53635

In [54]:
trainReviews = []
validReviews = []

for i in range(9*len(dataset)//10):
    review = re.sub('[^a-zA-Z]', ' ', dataset[i]['question']).lower().split()
    review = ' '.join([lemmatizer.lemmatize(r) for r in review])

    # removing stop word lower accuracy

    # review = ' '.join([lemmatizer.lemmatize(r) for r in review if not r in set(stopwords)])
    trainReviews.append(review)

for i in range(9*len(dataset)//10, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset[i]['question']).lower().split()
    review = ' '.join([lemmatizer.lemmatize(r) for r in review])

    # removing stop word lower accuracy

    # review = ' '.join([lemmatizer.lemmatize(r) for r in review if not r in set(stopwords)])
    validReviews.append(review)

vectorizer = TfidfVectorizer()

trainData = dataset[:9*len(dataset)//10]
validData = dataset[9*len(dataset)//10:]

Xtrain = vectorizer.fit_transform(trainReviews)
ytrain = [d['questionType'] for d in trainData]
Xvalid = vectorizer.transform(validReviews)
yvalid = [d['questionType'] for d in validData]


  (0, 16018)	0.20421939242900222
  (0, 15590)	0.15850453928292701
  (0, 15244)	0.12534758814099467
  (0, 14560)	0.07865583693076923
  (0, 14372)	0.1286155836554197
  (0, 13494)	0.22401998534741852
  (0, 11169)	0.4845875511398021
  (0, 7675)	0.276533286177792
  (0, 6961)	0.3763846262348915
  (0, 6757)	0.160794351309555
  (0, 6427)	0.10352885364240277
  (0, 5801)	0.1832780880753223
  (0, 4988)	0.22606919687509375
  (0, 4145)	0.08823871549583635
  (0, 2937)	0.4027164311244472
  (0, 2747)	0.13451632663923158
  (0, 486)	0.2867952350828493
  (1, 15197)	0.2830258055286051
  (1, 14560)	0.1508155843713247
  (1, 14372)	0.12330448168343848
  (1, 11104)	0.32074012238171296
  (1, 10247)	0.44924636076101787
  (1, 9687)	0.25751341990696847
  (1, 7653)	0.28873147290207907
  (1, 7491)	0.35193717100016436
  :	:
  (5957, 1193)	0.3112133506426025
  (5958, 16282)	0.30958286913513183
  (5958, 15255)	0.3131875896446626
  (5958, 15197)	0.25013191032934434
  (5958, 14509)	0.3009556842085689
  (5958, 14281)	0.0

In [55]:
print(validReviews)

['doe this come with a quick connect i have a gas karcher with a standard hose i want to use this an an extension ideally quick connect', 'how difficult is this item to put up for just one person', 'what is the shipping weight', 'doe this canopy come with the sidewall shown', 'what size is this one', 'used once top plastic bracket broke when the folded tent fell sideways from standing need repair what is the warranty', 'what are the dimension when canopy is closed for storage', 'can you print my logo on it', 'can you purchase cover for all side', 'doe this come with the one wall a pictured', 'are the x x inch dimension of the canopy when it is collapsed', 'what is the size of this gazebo length and width', 'what sise of base doe the gazebo need is x suffisant', 'can you screw the leg to a deck', 'is this gazebo actually round or oblong', 'what kind of fabric is this made of', 'can it be anchored to cement somehow to avoid it blowing away in the wind', 'is the square footage the same a 

In [ ]:
# model = linear_model.LinearRegression(fit_intercept=False)
model = linear_model.LogisticRegression(C=1.0)
model.fit(Xtrain, ytrain)
y_something = model.predict(Xvalid)
print(accuracy(y_something, yvalid))

0.8865771812080537


In [ ]:
# Naive Bayes

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
mod = MultinomialNB()
mod.fit(Xtrain, ytrain)
pred = mod.predict(Xvalid)
print(accuracy(pred, yvalid))

0.7686241610738255


In [ ]:
# K-Nearest Neighbor

In [ ]:
encoder = preprocessing.LabelEncoder()
mod = KNeighborsClassifier(n_neighbors=3)

mod.fit(Xtrain, ytrain)

pred = mod.predict(Xvalid)
print(accuracy(pred, yvalid))

0.6860738255033557


In [ ]:
# Popular Words

In [ ]:
for word in dataset:
  if word['questionType']  == 'yes/no':
    for i in word['question'].split():
      if i not in stopwords:
        wordCount[i] += 1
      # wordCount[i] += 1

In [ ]:
mostPopular = [(wordCount[i], i) for i in wordCount]
mostPopular.sort()
mostPopular.reverse()
print(mostPopular)

[(6024, 'I'), (3904, 'Does'), (3709, 'Is'), (3575, 'Will'), (3472, 'Can'), (2925, 'fit'), (2614, 'work'), (1917, 'use'), (1544, 'come'), (1270, 'used'), (1082, 'Do'), (901, 'need'), (876, 'one'), (856, 'Are'), (794, '?'), (770, 'like'), (754, 'product'), (724, 'Would'), (723, 'would'), (708, 'model'), (690, 'anyone'), (680, 'get'), (668, 'water'), (641, 'cover'), (617, 'want'), (596, 'The'), (552, 'pool'), (478, 'hose'), (464, '2'), (427, 'put'), (426, 'Has'), (422, 'grill'), (414, 'enough'), (403, 'battery'), (393, 'My'), (384, 'keep'), (383, 'it?'), (382, 'unit'), (378, 'pump'), (360, 'top'), (352, 'good'), (346, 'set'), (344, "I'm"), (342, 'replace'), (340, '&amp;'), (336, 'two'), (333, 'replacement'), (330, 'kill'), (327, 'without'), (309, 'x'), (297, 'inch'), (294, 'small'), (292, 'It'), (291, 'buy'), (284, 'part'), (282, 'make'), (282, 'hold'), (279, 'gas'), (278, 'side'), (275, 'table'), (270, 'says'), (263, 'made'), (262, 'well'), (261, 'filter'), (259, 'We'), (258, 'power'), (

In [ ]:
top10 = []
acc = defaultdict(int)
total = defaultdict(int)
data = defaultdict(float)

for i in range(len(mostPopular)):
  top10.append(mostPopular[i][1])

# top10 = ['Would', 'Will', 'Can', 'Is', 'Are', 'Do']

for word in dataset:
    for i in top10:
      if i in word['question'].split():
        if word['questionType']  == 'yes/no':
          acc[i] += 1

        total[i] += 1

for i in acc:
  # print(acc[i], " ", total[i])
  data[i] = float(acc[i]) / float(total[i]) * 100

# dict(sorted(data.items(), key=lambda item: item[1]))

# for i in data:
#   if data[i] > 60:
#     print(i, ": ", data[i])

# data = data[:10]

courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Top 10 Most Popular Word (without stop words)")
plt.ylabel("% of questions that are yes/no given it contains this word")
plt.show()

KeyboardInterrupt: ignored

In [ ]:
def feature(datum):
    #words picked by Leo and Kevin without looking at data, only use 1 element in feature vector
    counter = datum['question'].count("would") + datum['question'].count("will") + datum['question'].count("can") + datum['question'].count("is") + datum['question'].count("are") + datum['question'].count("do")
    feat = [1, datum['question'].count('?')]
    #words picked by Leo and Kevin without looking at data
    # feat = [1, datum['question'].count("would"), datum['question'].count("will"), datum['question'].count("can"), datum['question'].count("is"), datum['question'].count("are"), datum['question'].count("do")]
    # feat = [1, datum['question'].count("Would"), datum['question'].count("Will"), datum['question'].count("Can"), datum['question'].count("Is"), datum['question'].count("Are"), datum['question'].count("Do")]
    #remove stop words, get highest count, then hand pick likely options
    # feat = [1, datum['question'].count("Does"), datum['question'].count("Is"), datum['question'].count("Will"), datum['question'].count("Can"), datum['question'].count("Do")]
    return(feat)

In [ ]:
X = [feature(x) for x in dataset]
y = [d['questionType'] for d in dataset]

Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

In [ ]:
# model = linear_model.LinearRegression(fit_intercept=False)
model = linear_model.LogisticRegression(C=1.0)
model.fit(Xtrain, ytrain)

LogisticRegression()

In [ ]:
y_something = model.predict(Xvalid)

In [ ]:
print(accuracy(y_something, yvalid))

0.5714765100671141
